In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error,root_mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna


d:\Code files\Used Car Price Prediction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['KAGGLE_USERNAME']=os.getenv('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY']=os.getenv('KAGGLE_KEY')

In [3]:
import kagglehub

# This automatically downloads and unzips the dataset to ~/.kagglehub
path = kagglehub.dataset_download("adityadesai13/used-car-dataset-ford-and-mercedes")

print("Path to dataset files:", path)


Path to dataset files: C:\Users\aishw\.cache\kagglehub\datasets\adityadesai13\used-car-dataset-ford-and-mercedes\versions\3


In [4]:
os.listdir(path)

['audi.csv',
 'bmw.csv',
 'cclass.csv',
 'focus.csv',
 'ford.csv',
 'hyundi.csv',
 'merc.csv',
 'skoda.csv',
 'toyota.csv',
 'unclean cclass.csv',
 'unclean focus.csv',
 'vauxhall.csv',
 'vw.csv']

In [5]:
audi_path=os.path.join(path,'audi.csv')
bmw_path=os.path.join(path,'bmw.csv')
cclass_path=os.path.join(path,'cclass.csv')
focus_path=os.path.join(path,'focus.csv')
ford_path=os.path.join(path,'ford.csv')
hyundi_path=os.path.join(path,'hyundi.csv')
merc_path=os.path.join(path,'merc.csv')
skoda_path=os.path.join(path,'skoda.csv')
toyota_path=os.path.join(path,'toyota.csv')
unclean_cclass_path=os.path.join(path,'unclean cclass.csv')
unclean_focus_path=os.path.join(path,'unclean focus.csv')
vauxhall_path=os.path.join(path,'vauxhall.csv')
vw_path=os.path.join(path,'vw.csv')

In [6]:
audi_data=pd.read_csv(audi_path)
bmw_data=pd.read_csv(bmw_path)
cclass_data=pd.read_csv(cclass_path)
focus_data=pd.read_csv(focus_path)
ford_data=pd.read_csv(ford_path)
hyundi_data=pd.read_csv(hyundi_path)
merc_data=pd.read_csv(merc_path)
skoda_data=pd.read_csv(skoda_path)
toyota_data=pd.read_csv(toyota_path)
unclean_cclass_data=pd.read_csv(unclean_cclass_path)
unclean_focus_data=pd.read_csv(unclean_focus_path)
merc_data=pd.read_csv(merc_path)
vw_data=pd.read_csv(vw_path)

In [7]:
dfs = [
    audi_data,
    bmw_data,
    cclass_data,
    focus_data,
    ford_data,
    hyundi_data,
    merc_data,
    skoda_data,
    toyota_data,
    unclean_cclass_data,
    unclean_focus_data,
    vw_data
]

# Get the columns of the first DataFrame as a set
base_columns = set(dfs[0].columns)

# Check if all other DataFrames have the same columns
all_same_columns = all(set(df.columns) == base_columns for df in dfs)

print("All DataFrames have the same columns:", all_same_columns)

All DataFrames have the same columns: False


In [8]:
print(audi_data.columns)
print(bmw_data.columns)
print(cclass_data.columns)
print(focus_data.columns)
print(ford_data.columns)
print(hyundi_data.columns)
print(merc_data.columns)
print(skoda_data.columns)
print(toyota_data.columns)
print(unclean_cclass_data.columns)
print(unclean_focus_data.columns)
print(vw_data.columns)

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType',
       'engineSize'],
      dtype='object')
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType',
       'engineSize'],
      dtype='object')
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType',
       'tax(£)', 'mpg', 'engineSize'],
      dtype='object')
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      d

In [9]:
hyundi_data=hyundi_data.rename(columns={'tax(£)':'tax'})
hyundi_data


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,I20,2017,7999,Manual,17307,Petrol,145,58.9,1.2
1,Tucson,2016,14499,Automatic,25233,Diesel,235,43.5,2.0
2,Tucson,2016,11399,Manual,37877,Diesel,30,61.7,1.7
3,I10,2016,6499,Manual,23789,Petrol,20,60.1,1.0
4,IX35,2015,10199,Manual,33177,Diesel,160,51.4,2.0
...,...,...,...,...,...,...,...,...,...
4855,I30,2016,8680,Manual,25906,Diesel,0,78.4,1.6
4856,I40,2015,7830,Manual,59508,Diesel,30,65.7,1.7
4857,I10,2017,6830,Manual,13810,Petrol,20,60.1,1.0
4858,Tucson,2018,13994,Manual,23313,Petrol,145,44.8,1.6


## Fill the columns with NAN if not present

In [10]:
columns_to_add=[focus_data,cclass_data]
columns=['tax','mpg']
for df in columns_to_add:
    for col in columns:
        if col not in df.columns:
            df[col]=np.nan

In [11]:
dfs = [
    audi_data,
    bmw_data,
    cclass_data,
    focus_data,
    ford_data,
    hyundi_data,
    merc_data,
    skoda_data,
    toyota_data,
    vw_data
]

# Get the columns of the first DataFrame as a set
base_columns = set(dfs[0].columns)

# Check if all other DataFrames have the same columns
all_same_columns = all(set(df.columns) == base_columns for df in dfs)

print("All DataFrames have the same columns:", all_same_columns)

All DataFrames have the same columns: True


In [12]:
data=pd.concat(dfs,ignore_index=True)
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0


In [13]:
data.shape

(94908, 9)

In [14]:
data.isnull().sum()

model              0
year               0
price              0
transmission       0
mileage            0
fuelType           0
tax             9353
mpg             9353
engineSize         0
dtype: int64

In [15]:
data['tax']=data['tax'].fillna(value=data['tax'].median())
data['mpg']=data['mpg'].fillna(value=data['mpg'].median())

data.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [16]:
data.dtypes

model            object
year              int64
price             int64
transmission     object
mileage           int64
fuelType         object
tax             float64
mpg             float64
engineSize      float64
dtype: object

In [17]:
cat_cols=[col for col in data.columns if data[col].dtype==object]
df[cat_cols]

,model,transmission,fuelType
0,C Class,Automatic,Diesel
1,C Class,Automatic,Petrol
2,C Class,Automatic,Diesel
3,C Class,Automatic,Diesel
4,C Class,Automatic,Diesel
...,...,...,...
3894,C Class,Manual,Diesel
3895,C Class,Automatic,Diesel
3896,C Class,Manual,Diesel
3897,C Class,Automatic,Diesel


In [18]:
unique={}
for col in cat_cols:
    unique[col]=data[col].unique()

In [19]:
unique

{'model': array([' A1', ' A6', ' A4', ' A3', ' Q3', ' Q5', ' A5', ' S4', ' Q2',
        ' A7', ' TT', ' Q7', ' RS6', ' RS3', ' A8', ' Q8', ' RS4', ' RS5',
        ' R8', ' SQ5', ' S8', ' SQ7', ' S3', ' S5', ' A2', ' RS7',
        ' 5 Series', ' 6 Series', ' 1 Series', ' 7 Series', ' 2 Series',
        ' 4 Series', ' X3', ' 3 Series', ' X5', ' X4', ' i3', ' X1', ' M4',
        ' X2', ' X6', ' 8 Series', ' Z4', ' X7', ' M5', ' i8', ' M2',
        ' M3', ' M6', ' Z3', ' C Class', ' Focus', ' Fiesta', ' Puma',
        ' Kuga', ' EcoSport', ' C-MAX', ' Mondeo', ' Ka+',
        ' Tourneo Custom', ' S-MAX', ' B-MAX', ' Edge', ' Tourneo Connect',
        ' Grand C-MAX', ' KA', ' Galaxy', ' Mustang',
        ' Grand Tourneo Connect', ' Fusion', ' Ranger', ' Streetka',
        ' Escort', ' Transit Tourneo', ' I20', ' Tucson', ' I10', ' IX35',
        ' I30', ' I40', ' Ioniq', ' Kona', ' Veloster', ' I800', ' IX20',
        ' Santa Fe', ' Accent', ' Terracan', ' Getz', ' Amica', ' SLK',
        '

In [20]:
##One hot encoding
ohe_cols=['transmission','fuelType']
import category_encoders as ce
ohe=ce.OneHotEncoder(ohe_cols,return_df=True,handle_unknown='return_nan',use_cat_names=True)
data_encoded=ohe.fit_transform(data[ohe_cols])
data_encoded


,transmission_Manual,transmission_Automatic,transmission_Semi-Auto,transmission_Other,fuelType_Petrol,fuelType_Diesel,fuelType_Hybrid,fuelType_Other,fuelType_Electric
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
94903,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
94904,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
94905,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
94906,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [21]:
data_clean=pd.concat([data,data_encoded],axis=1)
data_clean.drop(columns=ohe_cols,inplace=True)
data_clean

,model,year,price,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Automatic,transmission_Semi-Auto,transmission_Other,fuelType_Petrol,fuelType_Diesel,fuelType_Hybrid,fuelType_Other,fuelType_Electric
0,A1,2017,12500,15735,150.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,A6,2016,16500,36203,20.0,64.2,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,A1,2016,11000,29946,30.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,A4,2017,16800,25952,145.0,67.3,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,A3,2019,17300,1998,145.0,49.6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94903,Eos,2012,5990,74000,125.0,58.9,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
94904,Fox,2008,1799,88102,145.0,46.3,1.2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
94905,Fox,2009,1590,70000,200.0,42.0,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
94906,Fox,2006,1250,82704,150.0,46.3,1.2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Converting model column into numerical values using target encoding

In [22]:
target_encoder=ce.TargetEncoder(cols=['model'],handle_unknown='return_nan')
encoded_values=target_encoder.fit_transform(data_clean['model'],data_clean['price'])
data_clean['model_encoded']=encoded_values
data_clean['model_encoded'].head()

0    14327.750557
1    22695.385027
2    14327.750557
3    20255.450398
4    17408.522032
Name: model_encoded, dtype: float64

In [23]:
data_clean.drop(columns=['model'],inplace=True)
data_clean

,year,price,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Automatic,transmission_Semi-Auto,transmission_Other,fuelType_Petrol,fuelType_Diesel,fuelType_Hybrid,fuelType_Other,fuelType_Electric,model_encoded
0,2017,12500,15735,150.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14327.750557
1,2016,16500,36203,20.0,64.2,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22695.385027
2,2016,11000,29946,30.0,55.4,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14327.750557
3,2017,16800,25952,145.0,67.3,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20255.450398
4,2019,17300,1998,145.0,49.6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17408.522032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94903,2012,5990,74000,125.0,58.9,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15421.685438
94904,2008,1799,88102,145.0,46.3,1.2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15118.926814
94905,2009,1590,70000,200.0,42.0,1.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15118.926814
94906,2006,1250,82704,150.0,46.3,1.2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15118.926814


In [24]:
x=data_clean.drop('price',axis=1).values
y=data_clean['price'].values

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [26]:
lr_model=LinearRegression()
lr_model.fit(x_train,y_train)

LinearRegression()

In [27]:
y_pred=lr_model.predict(x_test)
y_pred

array([11896.17944594, 14343.8547874 , 28619.22610906, ...,
       10400.48620679,  8420.2603188 , 11931.64425702], shape=(18982,))

In [28]:
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
rmse=root_mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')


MSE: 16882629.82091421
MAE: 2580.0077759000856
RMSE: 4108.84774856823
R2 Score: 0.8284849907006201


## Implementing Lasso Regression

In [29]:
lasso_model=Lasso(random_state=42)
lasso_model.fit(x_train,y_train)

Lasso(random_state=42)

In [30]:
y_pred=lasso_model.predict(x_test)
y_pred

array([11895.47984293, 14347.94100882, 28594.59760491, ...,
       10410.3970734 ,  8439.44614227, 11935.55075909], shape=(18982,))

In [31]:
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
rmse=root_mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

MSE: 16873087.607387576
MAE: 2579.8324416352184
RMSE: 4107.68640567748
R2 Score: 0.8285819325194669


## Ridge Regression

In [32]:
ridge_model=Ridge(random_state=42)
ridge_model.fit(x_train,y_train)

Ridge(random_state=42)

In [33]:
y_pred=ridge_model.predict(x_test)
y_pred

array([11895.75726507, 14344.32559717, 28616.0624352 , ...,
       10401.6585806 ,  8422.74186264, 11931.91975522], shape=(18982,))

In [34]:
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
rmse=root_mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

MSE: 16879489.862474382
MAE: 2579.987776097364
RMSE: 4108.46563360026
R2 Score: 0.8285168903517242


## ElasticNet

In [35]:
elasticnet_model=ElasticNet(random_state=42)
elasticnet_model.fit(x_train,y_train)

ElasticNet(random_state=42)

In [36]:
y_pred=elasticnet_model.predict(x_test)
y_pred

array([13202.33483068, 13478.22951127, 29729.20022319, ...,
       10612.80020784,  9383.17908136, 12140.07957042], shape=(18982,))

In [37]:
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
rmse=root_mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

MSE: 20523437.77313367
MAE: 2805.449074225509
RMSE: 4530.280098750371
R2 Score: 0.791497079669804


## Implementing Hyperparameter tuning for lasso regression

In [ ]:
def objective(trial):
    alpha=trial.suggest_float('alpha',1e-4,10.0,log=True)
    model=Lasso(alpha=alpha,random_state=42,max_iter=10000)
    
    score=cross_val_score(model,x_train,y_train,scoring='neg_mean_squared_error',cv=5,n_jobs=-1).mean()
    return -score
study=optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=50,n_jobs=1)
best_params=study.best_params
best_value=study.best_value
print(f'Best parameters: {best_params}')
print(f'Best value: {best_value}')
    

[I 2025-05-10 22:16:55,057] A new study created in memory with name: no-name-44ef604e-ddbd-4e89-8264-d5b7795d10ba
[I 2025-05-10 22:17:58,313] Trial 0 finished with value: 17295846.523859505 and parameters: {'alpha': 3.79993518335847}. Best is trial 0 with value: 17295846.523859505.
[I 2025-05-10 22:18:04,492] Trial 1 finished with value: 17295551.57533575 and parameters: {'alpha': 0.632414882369195}. Best is trial 1 with value: 17295551.57533575.
[I 2025-05-10 22:19:16,209] Trial 2 finished with value: 17291639.012714364 and parameters: {'alpha': 0.0035146221691106094}. Best is trial 2 with value: 17291639.012714364.
[I 2025-05-10 22:19:20,329] Trial 3 finished with value: 17295848.137675766 and parameters: {'alpha': 3.8285094846926997}. Best is trial 2 with value: 17291639.012714364.
[I 2025-05-10 22:20:55,509] Trial 4 finished with value: 17291625.82582298 and parameters: {'alpha': 0.0012748615837471482}. Best is trial 4 with value: 17291625.82582298.
[I 2025-05-10 22:22:00,663] Tria

Best parameters: {'alpha': 0.0001590613356750245}
Best value: 17291618.35901705


In [38]:
rf_model=RandomForestRegressor()
rf_model.fit(x_train,y_train)

RandomForestRegressor()

In [ ]:
y_pred=rf_model.predict(x_test)
y_pred